In [ ]:
import pandas as pd

test = pd.read_csv('/scratch/jribeiro/ec_number_prediction/final_data/test.csv')

In [ ]:
# now open everything
import pickle

with open('blast_f1_score.pkl', 'rb') as f:
    blast_f1_score = pickle.load(f)

with open('blast_recall_score.pkl', 'rb') as f:
    blast_recall_score = pickle.load(f)

with open('blast_precision_score.pkl', 'rb') as f:
    blast_precision_score = pickle.load(f)
# now open everything
import pickle

with open('esm2_t36_3B_UR50D_f1_score.pkl', 'rb') as f:
    esm2_t36_3B_UR50D_f1_score_list = pickle.load(f)

with open('esm2_t36_3B_UR50D_recall_score.pkl', 'rb') as f:
    esm2_t36_3B_UR50D_recall_score_list = pickle.load(f)

with open('esm2_t36_3B_UR50D_precision_score.pkl', 'rb') as f:
    esm2_t36_3B_UR50D_precision_score_list = pickle.load(f)
# do this for protein bert
import pickle

with open('protein_bert_f1_score.pkl', 'rb') as f:
    protein_bert_f1_score_list = pickle.load(f)

with open('protein_bert_recall_score.pkl', 'rb') as f:
    protein_bert_recall_score_list = pickle.load(f)

with open('protein_bert_precision_score.pkl', 'rb') as f:
    protein_bert_precision_score_list = pickle.load(f)
# now open everything
import pickle

with open('esm1b_f1_score.pkl', 'rb') as f:
    esm1b_f1_score_list = pickle.load(f)

with open('esm1b_recall_score.pkl', 'rb') as f:
    esm1b_recall_score_list = pickle.load(f)

with open('esm1b_precision_score.pkl', 'rb') as f:
    esm1b_precision_score_list = pickle.load(f)
# now open everything
import pickle

with open('deepec_f1_score.pkl', 'rb') as f:
    deepec_f1_score_list = pickle.load(f)

with open('deepec_recall_score.pkl', 'rb') as f:
    deepec_recall_score_list = pickle.load(f)

with open('deepec_precision_score.pkl', 'rb') as f:
    deepec_precision_score_list = pickle.load(f)
# now open everything
import pickle

with open('dspace_f1_score.pkl', 'rb') as f:
    dspace_f1_score_list = pickle.load(f)

with open('dspace_recall_score.pkl', 'rb') as f:
    dspace_recall_score_list = pickle.load(f)

with open('dspace_precision_score.pkl', 'rb') as f:
    dspace_precision_score_list = pickle.load(f)

In [ ]:
import pandas as pd

labels = pd.read_csv("/scratch/jribeiro/ec_number_prediction/final_data/train.csv", nrows=2).columns[8:]

In [ ]:
def get_best_above_threshold(metric_values, threshold):
    best_above_threshold = []
    for i, value in enumerate(metric_values):
        if threshold < 0 and value < threshold:
            best_above_threshold.append(i)
        elif threshold > 0 and value > threshold:
            best_above_threshold.append(i)
    return best_above_threshold

In [ ]:
import numpy as np
test = test.iloc[:, 8:]
num_labels = test.shape[1]
samples_per_label_test = np.zeros(num_labels, dtype=int)

for label in range(num_labels):
    samples_per_label_test[label] = np.sum(test.iloc[:, label])

In [ ]:
def get_results_for_threshold(comparison, samples_per_label_test, f1_score_list, method, blast=False):
    results = {}
    results["method"] = []
    results["threshold"] = []
    results["EC number"] = []
    results["Number of positives"] = []
    results["F1 score"] = []
    results["Difference of F1 score"] = []
    for threshold in [0.000000000000000000001, 0.1, 0.2, 0.3]:
        for i in get_best_above_threshold(comparison, threshold):
            results["method"].append(method)
            results["threshold"].append(threshold)
            results["EC number"].append(labels[i])
            results["Number of positives"].append(samples_per_label_test[i])
            results["F1 score"].append(np.array(f1_score_list)[i])
            results["Difference of F1 score"].append(comparison[i])
        
        if blast:
            for i in get_best_above_threshold(comparison, -threshold):
                results["method"].append(f"{method} - BLASTp")
                results["threshold"].append(threshold)
                results["EC number"].append(labels[i])
                results["Number of positives"].append(samples_per_label_test[i])
                results["F1 score"].append(np.array(blast_f1_score)[i])
                results["Difference of F1 score"].append(comparison[i])

    return results

In [ ]:
def get_comparison_results(model_f1_list, blast_f1_list):
    comparison = []
    for i in range(len(blast_f1_list)):
        comparison.append(model_f1_list[i] - blast_f1_list[i])
    return comparison

comparison_3b = get_comparison_results(esm2_t36_3B_UR50D_f1_score_list, blast_f1_score)
comparison_esm1b = get_comparison_results(esm1b_f1_score_list, blast_f1_score)
comparison_protein_bert = get_comparison_results(protein_bert_f1_score_list, blast_f1_score)

In [ ]:
import pandas as pd

results = pd.DataFrame(get_results_for_threshold(comparison_3b, samples_per_label_test, esm2_t36_3B_UR50D_f1_score_list, "DNN ESM2 3B"))

In [ ]:
results = pd.concat((results, pd.DataFrame(
    get_results_for_threshold(comparison_esm1b, samples_per_label_test, esm1b_f1_score_list, "DNN ESM1b"),
    )))
results = pd.concat((results, pd.DataFrame(
    get_results_for_threshold(comparison_protein_bert, samples_per_label_test, protein_bert_f1_score_list,
                              "DNN ProtBERT"))))
results.to_csv("results_for_threshold_by_ec_by_method.csv", index=False)
import pandas as pd

results = pd.DataFrame(
    get_results_for_threshold(comparison_3b, samples_per_label_test, esm2_t36_3B_UR50D_f1_score_list, "DNN ESM2 3B",
                              blast=True))
results = pd.concat((results, pd.DataFrame(
    get_results_for_threshold(comparison_esm1b, samples_per_label_test, esm1b_f1_score_list, "DNN ESM1b", blast=True),
    )))
results = pd.concat((results, pd.DataFrame(
    get_results_for_threshold(comparison_protein_bert, samples_per_label_test, protein_bert_f1_score_list,
                              "DNN ProtBERT", blast=True))))
results.to_csv("results_for_threshold_by_ec_by_method_with_blast.csv", index=False)

In [ ]:
aldo_keto_reductases = results[results["EC number"].isin(["1.1.1.107", "1.1.1.112", "1.1.1.116", "1.1.1.117", "1.1.1.122", 
                                                          "1.1.1.149", "1.1.1.156", "1.1.1.184", "1.1.1.188", "1.1.1.189", 
                                                          "1.1.1.19", "1.1.1.190", "1.1.1.191", "1.1.1.2", "1.1.1.20", "1.1.1.200",
                                                          "1.1.1.209", "1.1.1.21", "1.1.1.210", "1.1.1.216", "1.1.1.218", "1.1.1.225",
                                                          "1.1.1.239", "1.1.1.247", "1.1.1.263", "1.1.1.274", "1.1.1.283", "1.1.1.285",
                                                          "1.1.1.300", "1.1.1.307", "1.1.1.316", "1.1.1.317", "1.1.1.334", "1.1.1.346",
                                                          "1.1.1.353", "1.1.1.357", "1.1.1.358", "1.1.1.365", "1.1.1.372", "1.1.1.432", "1.1.1.50",
                                                          "1.1.1.51", "1.1.1.53", "1.1.1.54", "1.1.1.62", "1.1.1.64", "1.1.1.65", "1.1.1.91",
                                                          ])]
aldo_keto_reductases

In [ ]:
aldo_keto_reductases[aldo_keto_reductases["Difference of F1 score"] > 0].sort_values(by="Difference of F1 score", ascending=False)

In [ ]:
len(aldo_keto_reductases[aldo_keto_reductases["Difference of F1 score"] > 0]["EC number"].unique())

In [ ]:
len(aldo_keto_reductases["EC number"].unique())

In [ ]:
aldo_keto_reductases[aldo_keto_reductases["Difference of F1 score"] < 0]

In [ ]:
metal_dependent_hydrolases = results[results["EC number"].isin(["3.5.1.25", "3.5.1.5", "3.5.1.81", "3.5.1.82", "3.5.1.83", "3.5.1.91", "3.5.2.18",
                                                                "3.5.2.2", "3.5.2.3", "3.5.2.5", "3.5.2.7", "3.5.3.13", "3.5.4.1", "3.5.4.11", "3.5.4.2",
                                                                "3.5.4.28", "3.5.4.3", "3.5.4.31", "3.5.4.32", "3.5.4.4", "3.5.4.40", "3.5.4.41", "3.5.4.42", "3.5.4.43",
                                                                "3.5.4.45"]
                                                          )]
metal_dependent_hydrolases

In [ ]:
metal_dependent_hydrolases[metal_dependent_hydrolases["Difference of F1 score"] > 0].sort_values(by="Difference of F1 score", ascending=False)

In [ ]:
len(metal_dependent_hydrolases[metal_dependent_hydrolases["Difference of F1 score"] > 0]["EC number"].unique())

In [ ]:
len(metal_dependent_hydrolases["EC number"].unique())

In [ ]:
metal_dependent_hydrolases[metal_dependent_hydrolases["Difference of F1 score"] < 0].sort_values(by="Difference of F1 score", ascending=False)

In [ ]:
epoxide_hydrolases = results[results["EC number"].isin(["2.3.1.51", "3.1.1.1", "3.1.1.2", "3.1.1.23", "3.1.1.3", "3.1.1.35", "3.1.1.85", "3.1.1.89",
                                   "3.1.3.76", "3.3.2.10", "3.3.2.9", "3.7.1.14", "3.7.1.17", "3.7.1.25", "3.7.1.8", "3.7.1.9", 
                                   "3.8.1.3", "3.8.1.5"])]

In [ ]:
epoxide_hydrolases[epoxide_hydrolases["Difference of F1 score"] > 0]["EC number"].unique()

In [ ]:
len(epoxide_hydrolases[epoxide_hydrolases["Difference of F1 score"] < 0]["EC number"].unique())

In [ ]:
results[results["EC number"]=="2.3.3.1"]

In [ ]:
results[results["EC number"].isin(["2.3.3.1", "2.3.3.16", "2.3.3.5", "2.3.3.8"])]

In [ ]:
results[results["EC number"]=="2.6.1.44"]

In [ ]:
results[results["EC number"].isin(["2.6.1.52", "2.6.1.45", "2.6.1.51", "2.6.1.44", "2.6.1.112", "2.6.1.30", "2.6.1.35", "2.6.1.37", 
                                   "2.6.1.63"])]

In [ ]:
with open("amino_transferases", "r") as f:
    amino_transferases = f.read().splitlines()

amino_transferases = [x.strip() for x in amino_transferases]

amino_transferases = results[results["EC number"].isin(amino_transferases)]
len(amino_transferases[amino_transferases["Difference of F1 score"] < 0]["EC number"].unique())

In [ ]:
len(amino_transferases[amino_transferases["Difference of F1 score"] > 0]["EC number"].unique())

In [ ]:
amino_transferases[amino_transferases["Difference of F1 score"] > 0].sort_values(by="Difference of F1 score", ascending=False)

In [ ]:
len(amino_transferases["EC number"].unique())